# ENCODING

In [38]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns
# Gráficos
# ==============================================================================
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor
# Asunciones y Preprocesamiento
# ==============================================================================
from scipy import stats
import math
from scipy.stats import levene
import researchpy as rp
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
import itertools
# ANOVA
# ==============================================================================
import statsmodels.api as sm
from statsmodels.formula.api import ols
#Linear Regresion con Sklearn
# ==============================================================================
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')
# Tamaño gráficas
# ==============================================================================
plt.rcParams["figure.figsize"] = (10,8)

In [39]:
df = pd.read_pickle("datos/estandar_sin_vr.pkl")
df.head()

,date,season,year,month,holiday,weekday,workingday,weather,temp,hum,windspeed,casual,registered
0,2018-01-01,winter,0,1,holiday,Monday,weekend or holiday,cloudy,-0.486274,0.855582,-0.208909,331,654
1,2018-01-02,winter,0,1,not holiday,Tuesday,workingday,cloudy,-0.425690,0.332722,0.687550,131,670
2,2018-01-03,winter,0,1,not holiday,Wednesday,workingday,clear,-0.949970,-0.900337,0.685210,120,1229
3,2018-01-04,winter,0,1,not holiday,Thursday,workingday,clear,-0.938563,-0.170632,-0.210435,108,1454
4,2018-01-05,winter,0,1,not holiday,Friday,workingday,clear,-0.853992,-0.901843,0.060294,82,1518


## Para el map de workingday. 

In [40]:
estad = df.describe().T
estad

,count,mean,std,min,25%,50%,75%,max
temp,730.0,-0.011215,0.574404,-1.380508,-0.509150,1.361758e-16,0.490850,1.137258
hum,730.0,0.006678,0.678317,-2.983624,-0.506204,0.000000e+00,0.493796,1.649628
windspeed,730.0,0.096947,0.789169,-1.613788,-0.468363,0.000000e+00,0.531637,3.322433
casual,730.0,849.249315,686.479875,2.000000,316.250000,7.170000e+02,1096.500000,3410.000000
registered,730.0,3658.757534,1559.758728,20.000000,2502.250000,3.664500e+03,4783.250000,6946.000000


In [41]:
df_workingday = df.groupby('workingday')['casual', 'registered'].median()
df_workingday

,casual,registered
workingday,,
weekend or holiday,735.5,3720.5
workingday,711.5,3633.0


Haciendo la proporción entre las medias de las categorías

In [42]:
#mapa_wd_casual = {False: 1, True: (df_workingday.iloc[1,0] / df_workingday.iloc[0,0])}
#mapa_wd_registered = {False: (df_workingday.iloc[0,1] / df_workingday.iloc[1,1]), True: 1}
#print(f'Los valores del mapa de workingday para el modelo de casual sería: {mapa_wd_casual}')
#print(f'Los valores del mapa de workingday para el modelo de registered sería: {mapa_wd_registered}')

Considerando la media global para sacar las proporciones

In [43]:
# si ajustamos en base a las medias de cada categoría de 'workingday' respecto a la media global de esa columna paralos casos de 'casual' y 'registered':
mapa_wd_casual = {'weekend or holiday': (df_workingday.iloc[0,0] / estad.loc['casual', '50%']), 'workingday': (df_workingday.iloc[1,0] / estad.loc['casual', '50%'])}
mapa_wd_registered = {'weekend or holiday': (df_workingday.iloc[0,1] / estad.loc['registered', '50%']), 'workingday': (df_workingday.iloc[1,1] / estad.loc['registered', '50%'])}
print(f'Los valores del mapa de workingday para el modelo de casual sería: {mapa_wd_casual}')
print(f'Los valores del mapa de workingday para el modelo de registered sería: {mapa_wd_registered}')

Los valores del mapa de workingday para el modelo de casual sería: {'weekend or holiday': 1.0258019525801954, 'workingday': 0.9923291492329149}
Los valores del mapa de workingday para el modelo de registered sería: {'weekend or holiday': 1.0152817574021011, 'workingday': 0.9914040114613181}


## Para el map de season

In [44]:
df_season = df.groupby('season')['casual', 'registered'].median()
df_season

,casual,registered
season,,
autumn,536.0,3809.0
spring,863.0,3861.5
summer,1051.5,4110.5
winter,219.5,1855.5


In [45]:
mapa_se_casual = {'autumn': (df_season.iloc[0,0] / estad.loc['casual', '50%']), 'spring': (df_season.iloc[1,0] / estad.loc['casual', '50%']), 'summer': (df_season.iloc[2,0] / estad.loc['casual', '50%']), 'winter': (df_season.iloc[3,0] / estad.loc['casual', '50%'])}
mapa_se_registered = {'autumn': (df_season.iloc[0,1] / estad.loc['registered', '50%']), 'spring': (df_season.iloc[1,1] / estad.loc['registered', '50%']), 'summer': (df_season.iloc[2,1] / estad.loc['registered', '50%']), 'winter': (df_season.iloc[3,1] / estad.loc['registered', '50%'])}
print(f'Los valores del mapa de season para el modelo de casual sería: {mapa_se_casual}')
print(f'Los valores del mapa de season para el modelo de registered sería: {mapa_se_registered}')

Los valores del mapa de season para el modelo de casual sería: {'autumn': 0.7475592747559274, 'spring': 1.203626220362622, 'summer': 1.4665271966527196, 'winter': 0.30613668061366806}
Los valores del mapa de season para el modelo de registered sería: {'autumn': 1.039432391867922, 'spring': 1.053759039432392, 'summer': 1.1217082821667348, 'winter': 0.5063446582071224}


## Para la columna year

In [46]:
df_year = df.groupby('year')['casual', 'registered'].median()
df_year

,casual,registered
year,,
0,614.0,2915.0
1,905.0,4790.0


In [47]:
mapa_yr_casual = {0: (df_year.iloc[0,0] / estad.loc['casual', '50%']), 1: (df_year.iloc[1,0] / estad.loc['casual', '50%'])}
mapa_yr_registered = {0: (df_year.iloc[0,1] / estad.loc['registered', '50%']), 1: (df_year.iloc[1,1] / estad.loc['registered', '50%'])}
print(f'Los valores del mapa de year para el modelo de casual sería: {mapa_yr_casual}')
print(f'Los valores del mapa de year para el modelo de registered sería: {mapa_yr_registered}')

Los valores del mapa de year para el modelo de casual sería: {0: 0.8563458856345886, 1: 1.2622036262203626}
Los valores del mapa de year para el modelo de registered sería: {0: 0.7954700504843771, 1: 1.3071360349297312}


## Para holiday

In [48]:
df_holiday = df.groupby('holiday')['casual', 'registered'].median()
df_holiday

,casual,registered
holiday,,
holiday,1236.5,2774.5
not holiday,711.5,3694.5


In [49]:
mapa_hol_casual = {'holiday': (df_holiday.iloc[0,0] / estad.loc['casual', '50%']), 'not holiday': (df_holiday.iloc[1,0] / estad.loc['casual', '50%'])}
mapa_hol_registered = {'holiday': (df_holiday.iloc[0,1] / estad.loc['registered', '50%']), 'not holiday': (df_holiday.iloc[1,1] / estad.loc['registered', '50%'])}
print(f'Los valores del mapa de holiday para el modelo de casual sería: {mapa_hol_casual}')
print(f'Los valores del mapa de holiday para el modelo de registered sería: {mapa_hol_registered}')

Los valores del mapa de holiday para el modelo de casual sería: {'holiday': 1.7245467224546722, 'not holiday': 0.9923291492329149}
Los valores del mapa de holiday para el modelo de registered sería: {'holiday': 0.7571292127166053, 'not holiday': 1.0081866557511256}


## Para weekday

In [50]:
df_weekday = df.groupby('weekday')['casual', 'registered'].median()
df_weekday

,casual,registered
weekday,,
Friday,537.5,3963.5
Monday,1434.0,3077.0
Saturday,610.0,3834.5
Sunday,876.5,3715.5
Thursday,628.5,4052.5
Tuesday,982.0,2932.0
Wednesday,666.5,3700.0


In [51]:
mapa_day_casual = {'Monday': (df_weekday.iloc[1,0] / estad.loc['casual', '50%']), 'Thursday': (df_weekday.iloc[4,0] / estad.loc['casual', '50%']), 'Wednesday': (df_weekday.iloc[6,0] / estad.loc['casual', '50%']), 'Tuesday': (df_weekday.iloc[5,0] / estad.loc['casual', '50%']), 'Friday': (df_weekday.iloc[0,0] / estad.loc['casual', '50%']), 'Saturday': (df_weekday.iloc[2,0] / estad.loc['casual', '50%']), 'Sunday': (df_weekday.iloc[3,0] / estad.loc['casual', '50%'])}
mapa_day_registered = {'Monday': (df_weekday.iloc[1,1] / estad.loc['casual', '50%']), 'Thursday': (df_weekday.iloc[4,1] / estad.loc['casual', '50%']), 'Wednesday': (df_weekday.iloc[6,1] / estad.loc['casual', '50%']), 'Tuesday': (df_weekday.iloc[5,1] / estad.loc['casual', '50%']), 'Friday': (df_weekday.iloc[0,1] / estad.loc['casual', '50%']), 'Saturday': (df_weekday.iloc[2,1] / estad.loc['casual', '50%']), 'Sunday': (df_weekday.iloc[3,1] / estad.loc['casual', '50%'])}
print(f'Los valores del mapa de weekday para el modelo de casual sería: {mapa_day_casual}')
print(f'Los valores del mapa de weekday para el modelo de registered sería: {mapa_day_registered}')

Los valores del mapa de weekday para el modelo de casual sería: {'Monday': 2.0, 'Thursday': 0.8765690376569037, 'Wednesday': 0.9295676429567643, 'Tuesday': 1.3695955369595536, 'Friday': 0.7496513249651325, 'Saturday': 0.8507670850767085, 'Sunday': 1.2224546722454672}
Los valores del mapa de weekday para el modelo de registered sería: {'Monday': 4.291492329149233, 'Thursday': 5.652022315202231, 'Wednesday': 5.160390516039052, 'Tuesday': 4.089260808926081, 'Friday': 5.527894002789401, 'Saturday': 5.347977684797769, 'Sunday': 5.182008368200837}


## Para month

In [52]:
df_month = df.groupby('month')['casual', 'registered'].median()
df_month

,casual,registered
month,,
1,126.5,1657.5
2,202.5,2024.0
3,508.5,2347.0
4,832.5,3395.5
5,887.0,3883.5
6,968.5,4359.0
7,1045.0,4005.5
8,1111.0,4116.5
9,970.0,4162.0


In [53]:
mapa_month_casual = {1: (df_month.iloc[0,0] / estad.loc['casual', '50%']), 2: (df_month.iloc[1,0] / estad.loc['casual', '50%']), 3: (df_month.iloc[2,0] / estad.loc['casual', '50%']), 4: (df_month.iloc[3,0] / estad.loc['casual', '50%']), 5: (df_month.iloc[4,0] / estad.loc['casual', '50%']), 6: (df_month.iloc[5,0] / estad.loc['casual', '50%']), 7: (df_month.iloc[6,0] / estad.loc['casual', '50%']), 8: (df_month.iloc[7,0] / estad.loc['casual', '50%']), 9: (df_month.iloc[8,0] / estad.loc['casual', '50%']), 10: (df_month.iloc[9,0] / estad.loc['casual', '50%']), 11: (df_month.iloc[10,0] / estad.loc['casual', '50%']), 12: (df_month.iloc[11,0] / estad.loc['casual', '50%'])}
mapa_month_registered = {1: (df_month.iloc[0,1] / estad.loc['casual', '50%']), 2: (df_month.iloc[1,1] / estad.loc['casual', '50%']), 3: (df_month.iloc[2,1] / estad.loc['casual', '50%']), 4: (df_month.iloc[3,1] / estad.loc['casual', '50%']), 5: (df_month.iloc[4,1] / estad.loc['casual', '50%']), 6: (df_month.iloc[5,1] / estad.loc['casual', '50%']), 7: (df_month.iloc[6,1] / estad.loc['casual', '50%']), 8: (df_month.iloc[7,1] / estad.loc['casual', '50%']), 9: (df_month.iloc[8,1] / estad.loc['casual', '50%']), 10: (df_month.iloc[9,1] / estad.loc['casual', '50%']), 11: (df_month.iloc[10,1] / estad.loc['casual', '50%']), 12: (df_month.iloc[11,1] / estad.loc['casual', '50%'])}
print(f'Los valores del mapa de month para el modelo de casual sería: {mapa_month_casual}')
print(f'Los valores del mapa de month para el modelo de registered sería: {mapa_month_registered}')

Los valores del mapa de month para el modelo de casual sería: {1: 0.17642956764295675, 2: 0.2824267782426778, 3: 0.7092050209205021, 4: 1.1610878661087867, 5: 1.2370990237099024, 6: 1.3507670850767086, 7: 1.4574616457461647, 8: 1.5495118549511855, 9: 1.3528591352859136, 10: 1.0760111576011158, 11: 0.6262203626220363, 12: 0.40794979079497906}
Los valores del mapa de month para el modelo de registered sería: {1: 2.311715481171548, 2: 2.822873082287308, 3: 3.273361227336123, 4: 4.735704323570432, 5: 5.416317991631799, 6: 6.079497907949791, 7: 5.5864714086471405, 8: 5.741283124128312, 9: 5.804741980474198, 10: 5.598326359832636, 11: 5.225244072524407, 12: 4.400278940027894}


In [54]:
df_casual = df.drop('registered',axis=1)

In [55]:
df_casual.sample()

,date,season,year,month,holiday,weekday,workingday,weather,temp,hum,windspeed,casual
643,2019-10-06,autumn,1,10,not holiday,Sunday,weekend or holiday,clear,0.172551,0.180647,0.885845,3031


In [56]:
df_registered = df.drop('casual',axis=1)
df_registered.sample()

,date,season,year,month,holiday,weekday,workingday,weather,temp,hum,windspeed,registered
350,2018-12-17,autumn,0,12,not holiday,Monday,workingday,cloudy,-0.755557,-0.311664,0.632883,2464


In [57]:
df_casual['month'] = df_casual['month'].map(mapa_month_casual)

In [58]:
df_registered['month'] = df_registered['month'].map(mapa_month_registered)

In [59]:
df_casual['weekday'] = df_casual['weekday'].map(mapa_day_casual)
df_registered['weekday'] = df_registered['weekday'].map(mapa_day_registered)

In [60]:
df_casual['holiday'] = df_casual['holiday'].map(mapa_hol_casual)
df_registered['holiday'] = df_registered['holiday'].map(mapa_hol_registered)

In [61]:
df_casual['year'] = df_casual['year'].map(mapa_yr_casual)
df_registered['year'] = df_registered['year'].map(mapa_yr_registered)

In [62]:
df_casual['season'] = df_casual['season'].map(mapa_se_casual)
df_registered['season'] = df_registered['season'].map(mapa_se_registered)

In [63]:
df_casual['workingday'] = df_casual['workingday'].map(mapa_wd_casual)
df_registered['workingday'] = df_registered['workingday'].map(mapa_wd_registered)

In [64]:
df_registered

,date,season,year,month,holiday,weekday,workingday,weather,temp,hum,windspeed,registered
0,2018-01-01,0.506345,0.795470,2.311715,0.757129,4.291492,1.015282,cloudy,-0.486274,0.855582,-0.208909,654
1,2018-01-02,0.506345,0.795470,2.311715,1.008187,4.089261,0.991404,cloudy,-0.425690,0.332722,0.687550,670
2,2018-01-03,0.506345,0.795470,2.311715,1.008187,5.160391,0.991404,clear,-0.949970,-0.900337,0.685210,1229
3,2018-01-04,0.506345,0.795470,2.311715,1.008187,5.652022,0.991404,clear,-0.938563,-0.170632,-0.210435,1454
4,2018-01-05,0.506345,0.795470,2.311715,1.008187,5.527894,0.991404,clear,-0.853992,-0.901843,0.060294,1518
...,...,...,...,...,...,...,...,...,...,...,...,...
725,2019-12-27,0.506345,1.307136,4.400279,1.008187,5.527894,0.991404,cloudy,-0.768627,0.127049,1.721399,1867
726,2019-12-28,0.506345,1.307136,4.400279,1.008187,5.347978,1.015282,cloudy,-0.771244,-0.172705,-0.259536,2451
727,2019-12-29,0.506345,1.307136,4.400279,1.008187,5.182008,1.015282,cloudy,-0.771244,0.603476,-0.575896,1182
728,2019-12-30,0.506345,1.307136,4.400279,1.008187,4.291492,0.991404,clear,-0.763400,-0.680895,1.727718,1432


In [65]:
df_casual

,date,season,year,month,holiday,weekday,workingday,weather,temp,hum,windspeed,casual
0,2018-01-01,0.306137,0.856346,0.17643,1.724547,2.000000,1.025802,cloudy,-0.486274,0.855582,-0.208909,331
1,2018-01-02,0.306137,0.856346,0.17643,0.992329,1.369596,0.992329,cloudy,-0.425690,0.332722,0.687550,131
2,2018-01-03,0.306137,0.856346,0.17643,0.992329,0.929568,0.992329,clear,-0.949970,-0.900337,0.685210,120
3,2018-01-04,0.306137,0.856346,0.17643,0.992329,0.876569,0.992329,clear,-0.938563,-0.170632,-0.210435,108
4,2018-01-05,0.306137,0.856346,0.17643,0.992329,0.749651,0.992329,clear,-0.853992,-0.901843,0.060294,82
...,...,...,...,...,...,...,...,...,...,...,...,...
725,2019-12-27,0.306137,1.262204,0.40795,0.992329,0.749651,0.992329,cloudy,-0.768627,0.127049,1.721399,247
726,2019-12-28,0.306137,1.262204,0.40795,0.992329,0.850767,1.025802,cloudy,-0.771244,-0.172705,-0.259536,644
727,2019-12-29,0.306137,1.262204,0.40795,0.992329,1.222455,1.025802,cloudy,-0.771244,0.603476,-0.575896,159
728,2019-12-30,0.306137,1.262204,0.40795,0.992329,2.000000,0.992329,clear,-0.763400,-0.680895,1.727718,364


In [66]:
df_casual.isnull().sum()

date          0
season        0
year          0
month         0
holiday       0
weekday       0
workingday    0
weather       0
temp          0
hum           0
windspeed     0
casual        0
dtype: int64

In [67]:
df_registered.isnull().sum()

date          0
season        0
year          0
month         0
holiday       0
weekday       0
workingday    0
weather       0
temp          0
hum           0
windspeed     0
registered    0
dtype: int64

In [68]:
df_weather = df.groupby('weather')['casual', 'registered'].median()
df_weather

,casual,registered
weather,,
clear,829.0,3875.0
cloudy,535.0,3354.0
stormy,126.0,1672.0


In [69]:
mapa_wth_casual = {"clear": (df_weather.iloc[0,0] / estad.loc['casual', '50%']), "cloudy": (df_weather.iloc[1,0] / estad.loc['casual', '50%']), "stormy": (df_weather.iloc[2,0] / estad.loc['casual', '50%']) }
mapa_wth_registered = {"clear": (df_weather.iloc[0,1] / estad.loc['registered', '50%']), "cloudy": (df_weather.iloc[1,1] / estad.loc['registered', '50%']), "stormy": (df_weather.iloc[2,1] / estad.loc['casual', '50%'])}
print(f'Los valores del mapa de weather para el modelo de casual sería: {mapa_wth_casual}')
print(f'Los valores del mapa de weather para el modelo de registered sería: {mapa_wth_registered}')

Los valores del mapa de weather para el modelo de casual sería: {'clear': 1.1562064156206415, 'cloudy': 0.7461645746164575, 'stormy': 0.17573221757322174}
Los valores del mapa de weather para el modelo de registered sería: {'clear': 1.0574430345203985, 'cloudy': 0.9152681129758493, 'stormy': 2.331938633193863}


In [70]:
df_casual['weather'] = df_casual['weather'].map(mapa_wth_casual)
df_registered['weather'] = df_registered['weather'].map(mapa_wth_registered)

In [71]:
df_casual.to_pickle('datos/casual_cod_est.pkl')

In [72]:
df_registered.to_pickle('datos/registered_cod_est.pkl')